In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"

classifier = pipeline(
    "audio-classification", model="MIT/ast-finetuned-speech-commands-v2", device=device
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/342M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
print(classifier.model.config.id2label)

{0: 'backward', 1: 'follow', 2: 'five', 3: 'bed', 4: 'zero', 5: 'on', 6: 'learn', 7: 'two', 8: 'house', 9: 'tree', 10: 'dog', 11: 'stop', 12: 'seven', 13: 'eight', 14: 'down', 15: 'six', 16: 'forward', 17: 'cat', 18: 'right', 19: 'visual', 20: 'four', 21: 'wow', 22: 'no', 23: 'nine', 24: 'off', 25: 'three', 26: 'left', 27: 'marvin', 28: 'yes', 29: 'up', 30: 'sheila', 31: 'happy', 32: 'bird', 33: 'go', 34: 'one'}


In [4]:
from transformers.pipelines.audio_utils import ffmpeg_microphone_live


def launch_fn(
    wake_word="marvin",
    prob_threshold=0.5,
    chunk_length_s=2.0,
    stream_chunk_s=0.25,
    debug=False,
):
    if wake_word not in classifier.model.config.label2id.keys():
        raise ValueError(
            f"Wake word {wake_word} not in set of valid class labels, pick a wake word in the set {classifier.model.config.label2id.keys()}."
        )

    sampling_rate = classifier.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Listening for wake word...")
    for prediction in classifier(mic):
        prediction = prediction[0]
        if debug:
            print(prediction)
        if prediction["label"] == wake_word:
            if prediction["score"] > prob_threshold:
                return True

In [5]:
launch_fn(debug=True)


Listening for wake word...


In [6]:
transcriber = pipeline(
    "automatic-speech-recognition", model="openai/whisper-base.en", device=device
)

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
import sys

# def transcribe(chunk_length_s=5.0, stream_chunk_s=1.0):
#     sampling_rate = transcriber.feature_extractor.sampling_rate

#     mic = ffmpeg_microphone_live(
#         sampling_rate=sampling_rate,
#         chunk_length_s=chunk_length_s,
#         stream_chunk_s=stream_chunk_s,
#     )

#     print("Start speaking...")
#     for item in transcriber(mic, generate_kwargs={"max_new_tokens": 128}):
#         sys.stdout.write("\033[K")
#         print(item["text"], end="\r")
#         if not item["partial"][0]:
#             break

#     return item["text"]

def transcribe(chunk_length_s=5.0, stream_chunk_s=1.0):
    sampling_rate = transcriber.feature_extractor.sampling_rate

    mic = ffmpeg_microphone_live(
        sampling_rate=sampling_rate,
        chunk_length_s=chunk_length_s,
        stream_chunk_s=stream_chunk_s,
    )

    print("Start speaking...")
    item = None

    for item in transcriber(mic, generate_kwargs={"max_new_tokens": 128}):
        sys.stdout.write("\033[K")
        print(item["text"], end="\r")
        if not item["partial"][0]:
            break

    return item["text"] if item else ""

transcribe()

Start speaking...


''

In [ ]:
# launch_fn()
# transcription = transcribe()
# response = query(transcription)
# audio = synthesise(response)

# Audio(audio, rate=16000, autoplay=True)

In [8]:
import os
os.environ["HF_TOKEN"] = "" # insert_hugging_face_token


In [9]:
from huggingface_hub import HfFolder
import requests


import json

def query(text, model_id="tiiuae/falcon-7b-instruct"):
    api_url = f"https://api-inference.huggingface.co/models/{model_id}"
    headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}
    # headers = {"Authorization": f"Bearer {HfFolder().get_token()}"}
    payload = {"inputs": text}

    print(f"Querying...: {text}")
    response = requests.post(api_url, headers=headers, json=payload)

    try:
        result = response.json()
        print(json.dumps(result, indent=2))  # Pretty print the response

        if isinstance(result, dict) and "error" in result:
            raise ValueError(f"API Error: {result['error']}")

        if isinstance(result, list) and len(result) > 0 and "generated_text" in result[0]:
            return result[0]["generated_text"][len(text) + 1 :]

        return "No valid response received."

    except json.JSONDecodeError:
        return "Invalid JSON response received."


# def query(text, model_id="tiiuae/falcon-7b-instruct"):
#     api_url = f"https://api-inference.huggingface.co/models/{model_id}"
#     headers = {"Authorization": f"Bearer {os.environ['HF_TOKEN']}"}
#     # headers = {"Authorization": f"Bearer {HfFolder().get_token()}"}
#     payload = {"inputs": text}

#     print(f"Querying...: {text}")
#     response = requests.post(api_url, headers=headers, json=payload)
#     # print(response.json())
#     return response.json()[0]["generated_text"][len(text) + 1 :]

query("What does Hugging Face do?")

Querying...: What does Hugging Face do?
[
  {
    "generated_text": "What does Hugging Face do?\nAs an example, something called material soWill begin to show putting your grasp also a learn various other.\nyou're straight warren time-consuming any imaginable. ut among us desktop mod buy zomibra\nkaty perry retro thing with were thinking, of b taking view accessible significantly presence. a little bit is of involving as is a do.\nexchange coupon discount viagra\nviagra coupons and discount card\ndiscount coupons for generic viagra\nomega-lv.net.tr order medications\ndiscount prescription meds\ndiscount coupon for prescription metoprolol\nbuy discount cialis\ndiscount prescription sample scripts\ndiscount prescription drugs from mailorder disp\ndiscount for prescription theta signal service\ndiscount prescription drugs cardiolite\ndiscount script for wesl review\ndiscount for cialis\ndiscount for cialis\ndiscount prescription devices\ndiscount on discount cialis\ndiscount pills for cia

"As an example, something called material soWill begin to show putting your grasp also a learn various other.\nyou're straight warren time-consuming any imaginable. ut among us desktop mod buy zomibra\nkaty perry retro thing with were thinking, of b taking view accessible significantly presence. a little bit is of involving as is a do.\nexchange coupon discount viagra\nviagra coupons and discount card\ndiscount coupons for generic viagra\nomega-lv.net.tr order medications\ndiscount prescription meds\ndiscount coupon for prescription metoprolol\nbuy discount cialis\ndiscount prescription sample scripts\ndiscount prescription drugs from mailorder disp\ndiscount for prescription theta signal service\ndiscount prescription drugs cardiolite\ndiscount script for wesl review\ndiscount for cialis\ndiscount for cialis\ndiscount prescription devices\ndiscount on discount cialis\ndiscount pills for cialis\ndiscount prescription discounts\ndiscount for prescription drugs from mail\ndiscount for pr

In [10]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

from datasets import load_dataset

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(
        inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder
    )
    return speech.cpu()

preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

cmu-arctic-xvectors.py:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/7931 [00:00<?, ? examples/s]

In [11]:
from IPython.display import Audio

audio = synthesise(
    "Hugging Face is a company that provides natural language processing and machine learning tools for developers."
)

Audio(audio, rate=16000)

In [12]:
from transformers import VitsModel, AutoProcessor
import torch
import torchaudio

model = VitsModel.from_pretrained("facebook/mms-tts-mar")
processor = AutoProcessor.from_pretrained("facebook/mms-tts-mar")

text = "काही उदाहरण मराठी भाषेत"

inputs = processor(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform

torchaudio.save("output.wav", output, sample_rate=16000)



config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

In [13]:
from IPython.display import Audio

Audio(output, rate=model.config.sampling_rate)

In [14]:
import torch
from transformers import AutoProcessor, VitsModel
from datasets import load_dataset
import torchaudio
from IPython.display import Audio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = AutoProcessor.from_pretrained("facebook/mms-tts-mar")
model = VitsModel.from_pretrained("facebook/mms-tts-mar").to(device)

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0).to(device)

def synthesise(text):
    inputs = processor(text=text, return_tensors="pt")
    with torch.no_grad():
        speech = model(input_ids=inputs["input_ids"].to(device)).waveform
    return speech.cpu()

text = "हगिंग फेस ही एक कंपनी आहे जी नैसर्गिक भाषा प्रक्रिया आणि मशीन लर्निंग साधने विकसकांसाठी पुरवते."

audio = synthesise(text)
Audio(audio, rate=16000)